In [1]:
# 한국어 영어 번역 데이터셋
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import unicodedata

# 실제 프로젝트 AI hub 기타 등등..

# 1.
korean_sentences = [
    "안녕하세요",
    "오늘 날씨가 좋아요",
    "저는 학생입니다",
    "이것은 사과입니다",
    "고양이가 자고 있어요",
    "내일 비가 올까요",
    "저는 커피를 좋아해요",
    "그는 의사입니다",
    "이 책은 재미있어요",
    "우리는 친구예요"
]

english_sentences = [
    "hello",
    "the weather is nice today",
    "i am a student",
    "this is an apple",
    "the cat is sleeping",
    "will it rain tomorrow",
    "i like coffee",
    "he is a doctor",
    "this book is interesting",
    "we are friends"
]

In [2]:
# 텍스트 전처리
# unicode 정규화, 특수문자처리
# NFD  Normalization Form Decomposition -> 분해 가능한 모든 문자를 분해한다
import unicodedata
unicodedata.normalize('NFD',"e'")

"e'"

###### 같은 문장처럼 보이지만 내부적으로 다른 바이트 조합을 갖는 경우 -> 동일한 내부표현으로 바꿔주는 과정

- 유티코드
    - 전 세계의 모든 문자를 하나의 표준 코드로 표현하기 위한 체계

- 유티코드 정규화를 해야되는 이유
    - 문자 하나가 여러 방식으로 표현될 수 있기 때문이다.
    - 그 여러문자들을 하나의 기준으로 통합하고자 사용되고 있다.

$\acute{e}$  

단일문자 U+00E9  

$e$ + $\acute{}$  

$e$ + 악센트
###### ucicode_normalizer_md 참고

In [3]:
# 1.1
def preprocess_sentence(sentence, is_korean=False):
    '''
    Unicode 정규화, 특수문자처리
    Args :
        sentence  : 원본
        is_korean : 한국어 여부
    return :
        전처리된 문장
    '''
    
    sentence = unicodedata.normalize('NFD',sentence)
    if not is_korean:
        sentence = sentence.lower()
    
    # 정규식으로 특수문자 전후에 공백 추가
    # 안녕하세요! -> 안녕하세요 !
    # -> 공백을 추가하는 이유 ^
    # -> 토큰화를 쉽게 하기 위해, 공백을 기준으로 단어분리
    
    # r("[? . ! ,]")
    # 문자중에 ? . ! ,가  나오면 해당문자를 그룹으로 캡처한다. ^
    
    # r" \1 " 캡처한 문자 ( \1 ) 앞뒤로 공백을 하나씩 넣는다.
    sentence = re.sub(r"([? . ! ,])",r" \1 ",sentence)
    sentence = re.sub(r'[" "]'," ",sentence) # 이중 공백 처리
    
    # 시작 종료 토큰 추가
    sentence = '<start> ' + sentence + ' <end>' #^&
    return sentence

In [4]:
korean_processed = [preprocess_sentence(kor) for kor in korean_sentences]
english_processed = [preprocess_sentence(eng) for eng in english_sentences]

In [5]:
# 문장을 정수 시퀀스로 변환

# 1.2 토크나이저
def create_tokenizer(sentence):
    '''
    단어를 정수 인덱스로 변환
    Vocabulary 구축, word to index mapping
    Args :
        sentence : 문장 리스트
    returns :
        Tokenizer : keras Tokenizer 객체
    '''
    # 토크나이저로 단어사전 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        filters='', # 이미 전처리가 되었으므로 필터 비활성화
        oov_token='<UNK>'
    )
    # 각 문장(한국어 문장, 영어 문장)으로 단어사전 구축
    tokenizer.fit_on_texts(sentence)
    return tokenizer

In [6]:
# 한국어 영어 각각의 토크나이저 생성
korean_tokenizer = create_tokenizer(korean_processed)
# 토크나이저를 함수안이 아닌 글로벌 범위에서 생성한경우
english_tokenizer = create_tokenizer(english_processed)

# korean_tokenizer.word_index

# 단어사전크기 확인
# 단어사전 크기 확인하는 이유 ^
korean_vocab_size = len(korean_tokenizer.word_index) + 1
# +1 하는 이유 ^
# Tokenizer에는 인덱스가 1부터시작하는데,
# Embedding Layer나 모델에서 입력 차원을 설정할 때,
# 패딩(0번)을 포함할 수 있도록 공간을 확보하기 위함
english_vocab_size = len(english_tokenizer.word_index) + 1
print(f'한국어 단어사전 크기 : {korean_vocab_size}')
print(f'영어 단어사전 크기 : {english_vocab_size}')

한국어 단어사전 크기 : 27
영어 단어사전 크기 : 32


In [7]:
# 1.3 점수시퀀스 변경
def padding_sentences(tokenizer, sentences, maxlen):
    '''
    문장을 고정길이의 정수 시퀀스로 변환
    padding : 짧은 문장을 동일길이로 맞추는
    Args :
        tokenizer
        sentence : 전처리된 문장 리스트
        maxlen : 가장 긴 문장의 길이
    Retuens :
        패딩된 점수 시퀀스 배열 tf의 pad_sequnce
    '''
    # texts_to_sequences
    sequnece = tokenizer.texts_to_sequences(sentences) # [i love you] -> [1,5,7]
    # 길이 맞추기 (패딩 추가)
    # pad_sequences
    padded = tf.keras.preprocessing.sequence.pad_sequences(
        sequnece,
        maxlen = maxlen,
        padding = 'post'
    )
    return padded

#### 패딩 기법 pre / post 어느것을 선택 할지의 대한 기준
- 딥러닝 RNN / LSTM / GUR 전통적인 시계열 처리 기법
    - pre
    - RNN 앞에서부터 읽어 -> 실제 단어 뒤쪽에 몰려있다 -> hidden state 유효한 문맥을 포함 -> h_last에 정보가 압축
- Transformer 계열은 post
    - 위치정보 앞 -> 뒤 순서를 그대로 유지해야 자연스럽다 -> 뒤쪽 패딩은 .. masking 처리 ^


In [8]:
# 최대 시퀀스의 길이 조회 및 변수에 해당 값 저장 (가장 긴 문장기준)
max_korean_len = max(len(s.split()) for s in korean_processed)
max_english_len = max(len(s.split()) for s in english_processed)
print('시퀀스의 최대길이')
print(f'한국어 : {max_korean_len}')
print(f'영어 : {max_english_len}')


# 만들어 놓은 함수로 패딩 수행 : padding_sentence
korean_tensor = padding_sentences(korean_tokenizer,korean_processed,max_korean_len)
english_tensor = padding_sentences(english_tokenizer,english_processed,max_english_len)

print(f'인코딩 첫번째 문장')
print(f'한국어 패딩 문장 : {korean_processed[2]}')
print(f'영어 패딩 문장{english_processed[2]}')
print(f'한국어 패딩 문장 : {korean_tensor[2]}')
print(f'영어 패딩 문장{english_tensor[2]}')

###
# 1.4
BUFFER_SIZE = len(korean_tensor)
BATCH_SIZE = 2

# tensorflow Dataset 객체
# 배치 처리와 셔플
# 만든 데이터 2개를 묶어 모델에 한번에 넣기위한 데이터 셋 세팅
# 묶을때 batch사용
# from_tensor_slice ^
dataset = tf.data.Dataset.from_tensor_slices((korean_tensor, english_tensor))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True) #^
print('데이터 셋 준비완료')
print(f'전체샘플수 : {len(korean_tensor)}')
print(f'배치크기 : {BATCH_SIZE}')
print(f'배치 수 :{len(korean_tensor) // BATCH_SIZE}')
    # 배치 수 = 입력 데이테 갯수 // 배치사이즈

시퀀스의 최대길이
한국어 : 5
영어 : 7
인코딩 첫번째 문장
한국어 패딩 문장 : <start> 저는   학생입니다 <end>
영어 패딩 문장<start> i   am   a   student <end>
한국어 패딩 문장 : [2 4 9 3 0]
영어 패딩 문장[ 2  6 13  7 14  3  0]
데이터 셋 준비완료
전체샘플수 : 10
배치크기 : 2
배치 수 :5


In [9]:
# from_tensor_slice((korean_tensor, english_tensor)): 번역할 문장, 번역된 문장끼리 묶기
#   - 한국어/영어 텐서를 '샘플 단위'로 잘라서 Dataset으로 만듦
#   - (입력 1개, 정답 1개) 형태의 pair를 샘플 하나로 구성
#   - 즉, 각 문장 쌍이 하나의 학습 샘플이 됨

# shuffle(BUFFER_SIZE): 
#   - 전체 샘플 순서를 무작위로 섞어서
#   - 학습이 특정 순서에 치우치지 않도록 함
#   - 셔플을 얼마나 강하게 섞을 것인가의 파리미터에
#     텐서길이만큼의 숫자를 입력한다는 것은 완전 랜덤하게 섞겠다는 말

# batch(BATCH_SIZE, drop_remainder=True): 모델에 넣기위한 데이터 묶음 만들기
#   - 2개씩들어가게 해서 아까 묶어놓은 문장끼리 짝지어 잘 들어가도록 ^ 왜 두번그렇게 하지?
#   - 여러 샘플을 한 번에 묶어서 모델에 넣기 위한 batch 단위를 생성
#   - BATCH_SIZE=2 → 샘플 2개씩 묶어서 모델이 동시에 처리하도록 함
#   - drop_remainder=True → 마지막 배치가 2개보다 적으면 버려서
#                          모든 batch가 동일한 크기(2)를 유지하도록 함
#   - 안정적 학습과 텐서 shape 일관성을 위함

# 한 줄 요약:
#   from_tensor_slice는 "샘플 하나 만들기"
#   batch는 "샘플 여러 개 묶어서 모델에 넣기"

```
-- step2 --
seq2seq 구조
'''
Encoder
    입력문장 -> Embedding -> LSTM + Hidden States
Decoder
    <start> 토큰 -> Embedding -> LSTM + context -> 단어예측
    예측단어 -> 단어입력 -> 반복 (<end> 나올때 까지)
```

In [10]:
# 2-1
# Encoder 클래스
class Encoder(tf.keras.Model): # Model 상속
    '''
    입력문장을 고차원 벡터로 압축
    Embedding -> LSTM -> Hidden States 출력
    구조 :
        입력(정수시퀀스) -> Embedding -> LSTM 모든 타임스텝의 출력
    '''
    def __init__(self,vocab_size,embedding_dim,enc_units,batch_size):
        '''
        Args :
            vocab_size : 단어사전크기(임베딩 테이블 크기)
            embedding_dim : 임베딩 벡터 차원
            enc_units : LMS 유닛수(hidden state 차원)
            batch_seize : 배치크기
        '''
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units
        # Embedding Layer : 정수 -> 밀집 벡터(Dense Vector)
        # 학습 과정을 통해 단어 간 의미적 관계학습
        # -> 유사 단어는 가까운 벡터공간에 위치
        # 단점 : 훈련필요(사전학습 또는 임베딩을 학습)
        
        # 희소벡터 :
        # 대부분의 값이 0(대표적인 원-핫 벡터)
        # 차원이 크다
        # 단어간 충돌이 없다
        # 단어간 유사도 표현 못함
        
        # 밀집 벡터 :
        # 학습기반(Word2Vec Glovw Embedding)
        
        # 희소벡터 :
        # 규칙기반(원핫 Bow) - 초창기 자연어 모델
        
        # mask_zero = True ^
        # 패딩에 마스크 처리를 해서 모델이 해석하지 않게 한다. 
        # ^ 
        
        # embedding 레이어 생성
        self.embedding = tf.keras.layers.Embedding(
            vocab_size,embedding_dim, mask_zero = True #^
        )
        
        # LSTM  레이어 생성
        self.lstm = tf.keras.layers.LSTM(
            enc_units,
            return_sequences = True, # [batch, seq_len_units] ^
            return_state = True, # (output, h, c) ^
            recurrent_initializer = 'glorot_uniform' # 가중치 초기화 설정
        )
        
    def call(self, x, hidden):
        '''
        입력 시퀀스를 처리해서 hidden states 생성
        Args :
            x : 입력 시퀀스[ batch_size, seq_len]
            hidden : 초기 hidden state(첫 호출시 0벡터)
        Returns :
        '''
        # 위에서 만든 레이어를 가져와 인코딩
        # 1단계 : Embedding
        # [batch, seq_len]-> [batch,seq_len, embedding_dim]
        x = self.embedding(x)
        
        # 2단계 : LSTM 처리
        # output : 모든타임스텝의 출력(Attenton에서 사용)
        # state_h : 마지막 hidden state(디코더 초기화)
        # state_c : 마지막 cell state(LSTM 내부 메모리)
        output, state_h, state_c = self.lstm(x,initial_state=hidden)
        return output,state_h,state_c
    
    def initialize_hidden_state(self):
        '''
        인코더의 초기 hidden state 0 으로 초기화
        Returns :
            두개의 0텐서 [batch, enc_unit] ) (h와 c)
        '''
        return [
            tf.zeros((self.batch_size, self.enc_units)),
            tf.zeros((self.batch_size, self.enc_units))
        ]

In [11]:
# 위 코드 궁금한점 정리 :
# LSTM에서 Hidden State가 중요한 이유?
# 파이썬 콜백함수에 대하여?

In [12]:
# 2-2 Encoder 테스트
embedding_dim = 2
units = 512 # LSTM unit 수
# encoder 객체
encoder = Encoder(korean_vocab_size,embedding_dim,units,BATCH_SIZE)

# 샘플 데이터
sample_input_batch, sample_target_batch = next(iter(dataset))
initial_hidden = encoder.initialize_hidden_state()

# encorder 클래스에 forward-> call 호출해서 결과를 얻어야....
# why call호출을 하는지? 우리는 모델을 funtional API 방식
# 객체를 함수처럼 사용하면 된다.
sample_output, sample_h, sample_c = encoder(sample_input_batch,initial_hidden)

print(f'{sample_input_batch.shape}') # [batch, korean_seq_len]
print(f'{sample_target_batch.shape}') # [batch, english_seq_len]
print(f'{sample_h.shape}') # [batch,units]
print(f'{sample_c.shape}') # [batch,units]

(2, 5)
(2, 7)
(2, 512)
(2, 512)


- 인코더의 출력형태
    - 입력 (2, 3)
    - 출력 (2,5,512) -------> 모든 타임스텝의 hidden state(Attention에서 사용)
    - hidden state(2, 512) -->마지막 타임스탬프의 상태(디코더 초기화)
    - cell state : (2, 512) --> LSTM 내부메모리

Attention Layer 구현
1. Query 생성: 디코더의 현재 hidden state
2. Key/Value: 인코더의 모든 hidden states
3. Score 계산: Q · K^T (내적으로 유사도 측정)
4. Attention Weight: Softmax(Score) (확률 분포로 변환)
5. Context Vector: Weight로 Value를 가중합


In [13]:
# query, key, value, attention score, context vector
class BAttention(tf.keras.layers.Layer):
    # tf.keras.layers.Layer를 부모 클래스로 받는 이유 ^ 
    '''
    query : 무엇을 찾고 있는 지(디코더의 현재 상태)
    key : 어디에 정보가 있는지(인코더의 각 타임스텝)
    value : 가져올 내용(인코더의 실제 정보)
    '''
    def __init__(self,units):
        '''
        Args :
            units : Attention 레이어의 차원(보통 인코더 units와 동일)
        '''
        super(BAttention,self).__init__()
        # 쿼리 변환 행렬 디코더의 hidden_state를 attention 공간
        self.W1 = tf.keras.layers.Dense(units)
        # key 변환행렬 인코더의 hidden_state를 attention 공간
        self.W2 = tf.keras.layers.Dense(units)
        # V : socre 계산용 벡터
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        """
        Args:
            query: 디코더의 hidden state [batch, dec_units]
            values: 인코더의 모든 hidden states [batch, enc_seq_len, enc_units]
        
        Returns:
            context_vector: 가중합된 인코더 정보 [batch, enc_units]
            attention_weights: 각 타임스텝의 가중치 [batch, enc_seq_len, 1]
        
        동작 과정:
            1. Query와 Key를 같은 차원으로 변환
            2. Score 계산 (유사도 측정)
            3. Softmax로 확률 분포 생성
            4. 가중합으로 context vector 생성
        """
        
        # 1단계: Query 확장
        # [batch, dec_units] → [batch, 1, dec_units]
        # 이유: Key와 브로드캐스팅하기 위해 차원 추가
        query_with_time_axis = tf.expand_dims(query, 1)
        
        # 2단계: Score 계산 (Bahdanau 방식)
        # W1(query): [batch, 1, units]
        # W2(values): [batch, enc_seq_len, units]
        # 덧셈 브로드캐스팅: [batch, enc_seq_len, units]
        # tanh: 비선형 활성화 (-1 ~ 1 범위)
        # V: [batch, enc_seq_len, 1] (각 타임스텝의 점수)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)
        ))
        
        # 3단계: Attention Weights 계산
        # Softmax: 점수를 확률 분포로 변환 (합이 1)
        # [batch, enc_seq_len, 1]
        # 높은 점수 → 높은 가중치 (더 많이 참고)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # 4단계: Context Vector 계산
        # attention_weights: [batch, enc_seq_len, 1]
        # values: [batch, enc_seq_len, enc_units]
        # 곱셈: [batch, enc_seq_len, enc_units]
        # sum: [batch, enc_units] (가중합)
        # 의미: 중요한 타임스텝의 정보를 더 많이 가져옴
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights
attention_layer = BAttention(units)

#query : 디코더의 현재 상태(첫번째 디코딩 스텝)
sample_query = sample_h  
# values : 인코더의 모든 출력
sample_values = sample_output

context_vector, attention_weights = attention_layer(sample_query, sample_values)
print(f'Attention Weight 첫번재 샘플')
weight_example = attention_weights[0].numpy().flatten()
print(f'가중치 : {weight_example}')
print(f'합계 : {np.sum(weight_example):.4f}')
        

Attention Weight 첫번재 샘플
가중치 : [0.20018375 0.19998468 0.19985524 0.19989869 0.20007761]
합계 : 1.0000


> step 4 Context Vector 결합 및 디코더 구현

Decoder 각 타임스텝:

1. 입력 단어 → Embedding
2. Attention으로 Context Vector 계산
3. Embedding + Context Vector → 결합
4. LSTM으로 처리 → Hidden State 업데이트
5. Dense Layer → 다음 단어 확률 분포

In [14]:
# 디코더 클래스 생성
class Decoder(tf.keras.Model):
    """
    목적: Attention 메커니즘을 사용하는 디코더
    개념:
        - Teacher Forcing: 학습 시 정답을 입력으로 사용
        - Attention Context: 매 스텝마다 인코더 정보 참고
        - Output Layer: 단어 확률 분포 생성
    
    일반 Decoder vs Attention Decoder:
        - 일반: 인코더의 마지막 상태만 사용 (정보 손실)
        - Attention: 모든 인코더 상태를 동적으로 참고 (정보 보존)
    """
    
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        """
        Args:
            vocab_size: 출력 단어 사전 크기
            embedding_dim: 임베딩 벡터 차원
            dec_units: LSTM 유닛 수
            batch_sz: 배치 크기
        """
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        
        # Embedding Layer: 출력 단어를 벡터로 변환
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        
        # LSTM Layer: 시퀀스 생성
        # return_sequences=True: 다음 타임스텝을 위해 출력 유지
        # return_state=True: hidden state 업데이트
        self.lstm = tf.keras.layers.LSTM(
            dec_units,
            return_sequences=True,
            return_state=True,
            recurrent_initializer='glorot_uniform'
        )
        
        # Output Layer: 단어 확률 분포 생성
        # vocab_size 차원 → 각 단어의 로그 확률
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # Attention Layer
        self.attention = BAttention(dec_units)
    
    def call(self, x, hidden, enc_output):
        """
        목적: 한 타임스텝의 디코딩 수행
        
        Args:
            x: 입력 단어 [batch, 1] (Teacher Forcing)
            hidden: 이전 타임스텝의 hidden state [batch, dec_units]
            enc_output: 인코더의 모든 출력 [batch, enc_seq_len, enc_units]
        
        Returns:
            predictions: 다음 단어 확률 분포 [batch, vocab_size]
            state_h: 현재 hidden state [batch, dec_units]
            attention_weights: Attention 가중치 [batch, enc_seq_len, 1]
        
        동작 과정:
            입력 → Embedding → Attention → 결합 → LSTM → 출력
        """
        
        # 1단계: Attention으로 Context Vector 계산
        # hidden: [batch, dec_units] (Query)
        # enc_output: [batch, enc_seq_len, enc_units] (Key, Value)
        context_vector, attention_weights = self.attention(hidden, enc_output)
        
        # 2단계: 입력 단어 임베딩
        # [batch, 1] → [batch, 1, embedding_dim]
        x = self.embedding(x)
        
        # 3단계: 임베딩과 Context Vector 결합
        # context_vector: [batch, enc_units] → [batch, 1, enc_units]
        # x: [batch, 1, embedding_dim]
        # 결합: [batch, 1, embedding_dim + enc_units]
        # 의미: 현재 입력 + 인코더 정보를 함께 사용
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # 4단계: LSTM 처리
        # output: [batch, 1, dec_units]
        # state_h, state_c: [batch, dec_units]
        output, state_h, state_c = self.lstm(x)
        
        # 5단계: 차원 조정 및 출력 생성
        # [batch, 1, dec_units] → [batch, dec_units]
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # 6단계: 단어 확률 분포 생성
        # [batch, dec_units] → [batch, vocab_size]
        # Softmax는 loss 함수에서 처리 (수치 안정성)
        predictions = self.fc(output)
        
        return predictions, state_h, attention_weights

In [15]:
# 디코더 함수 적용
# 디코더 객체 생성
decoder = Decoder(english_vocab_size,embedding_dim,units,BATCH_SIZE)
# 샘플임력 (디코더의 첫 번째 입력은 <start> 토큰)
sample_decoder_input = tf.random.uniform((BATCH_SIZE,1))  # [batch, 1]
sample_decoder_hidden = sample_h # 인코더의 마지막 hidden state

predictions, dec_h, attn_weights = decoder(
    sample_decoder_input,
    sample_decoder_hidden,
    sample_output
)

print('Decoder 출력 형태')
print(f'입력 형태 : {sample_decoder_input.shape}')
print(f'prediction 형태 : {predictions.shape}')  # batch, vocab_size-->english_vocab_size
print(f'Hidden state 형태 : {dec_h.shape}')
print(f'attn_weights state 형태 : {attn_weights.shape}')

Decoder 출력 형태
입력 형태 : (2, 1)
prediction 형태 : (2, 32)
Hidden state 형태 : (2, 512)
attn_weights state 형태 : (2, 5, 1)


> step5 학습루프
```
Teacher Forcing:
학습 시: 입력으로 정답 사용 (빠른 수렴)
  디코더 입력: <start> → 정답단어1 → 정답단어2 → ...

추론 시: 입력으로 이전 예측 사용
  디코더 입력: <start> → 예측단어1 → 예측단어2 → ...
```
Loss 계산:
1. 예측 분포와 정답 비교 (Cross Entropy)
2. 패딩은 무시 (마스킹)
3. 역전파로 가중치 업데이트

In [16]:
# --- 5.1 옵티마이저와 손실 함수 ---
optimizer = tf.keras.optimizers.Adam()  # Adam: 적응형 학습률

# Sparse Categorical Crossentropy
# - Sparse: 정답이 원-핫이 아닌 정수 인덱스
# - from_logits=True: Softmax를 내부에서 처리 (수치 안정성)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'  # 샘플별 손실 계산 (마스킹용)
)

def loss_function(real, pred):
    """
    패딩을 무시하는 손실 함수
    개념: Masking (패딩 토큰은 loss 계산 제외)
    
    Args:
        real: 정답 시퀀스 [batch, seq_len]
        pred: 예측 확률 분포 [batch, seq_len, vocab_size]
    
    Returns:
        평균 손실 (스칼라)
    
    왜 마스킹이 필요한가?
        - 패딩(0)은 실제 단어가 아님
        - 패딩에 대한 loss는 학습을 방해함
        - 마스크로 패딩 부분의 loss를 0으로 설정
    """
    # 1단계: 각 샘플의 loss 계산
    # real: [batch, seq_len]
    # pred: [batch, seq_len, vocab_size]
    # loss: [batch, seq_len]
    loss_ = loss_object(real, pred)
    
    # 2단계: 패딩 마스크 생성
    # 0이 아닌 값 → 1.0 (실제 단어)
    # 0인 값 → 0.0 (패딩)
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.cast(mask, dtype=loss_.dtype)
    
    # 3단계: 마스크 적용
    # 패딩 위치의 loss를 0으로 만듦
    loss_ *= mask
    
    # 4단계: 평균 계산
    # 실제 단어 개수로만 나눔 (패딩 제외)
    return tf.reduce_mean(loss_)

# --- 5.2 체크포인트 설정 ---
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = checkpoint_dir + "/ckpt"
checkpoint = tf.train.Checkpoint(
    optimizer=optimizer,
    encoder=encoder,
    decoder=decoder
)

# --- 5.3 학습 스텝 함수 ---
@tf.function  # 그래프 모드로 컴파일 (속도 향상)
def train_step(inp, targ, enc_hidden):
    """
    한 배치의 학습 수행
    개념: Teacher Forcing, Gradient Descent
    
    Args:
        inp: 입력 시퀀스 (한국어) [batch, inp_seq_len]
        targ: 정답 시퀀스 (영어) [batch, targ_seq_len]
        enc_hidden: 인코더 초기 hidden state
    
    Returns:
        batch_loss: 배치 평균 손실
    
    Teacher Forcing:
        디코더 입력으로 정답 시퀀스 사용
        예: targ = [<start>, "the", "cat", <end>]
            입력: [<start>, "the", "cat"]
            정답: ["the", "cat", <end>]
    """
    loss = 0
    
    # GradientTape: 자동 미분을 위한 컨텍스트
    with tf.GradientTape() as tape:
        # 1단계: Encoder 실행
        enc_output, enc_h, enc_c = encoder(inp, enc_hidden)
        
        # 2단계: Decoder 초기 상태 설정
        dec_hidden = enc_h  # 인코더의 마지막 hidden state 사용
        
        # 3단계: Decoder 입력 준비 (<start> 토큰)
        # targ[:, 0]: 첫 번째 열 (모두 <start> 토큰)
        dec_input = tf.expand_dims(targ[:, 0], 1)  # [batch, 1]
        
        # 4단계: 각 타임스텝마다 디코딩
        for t in range(1, targ.shape[1]):
            # Decoder 실행
            predictions, dec_hidden, _ = decoder(
                dec_input,
                dec_hidden,
                enc_output
            )
            
            # Loss 계산
            # targ[:, t]: 현재 타임스텝의 정답 단어
            loss += loss_function(targ[:, t], predictions)
            
            # Teacher Forcing: 정답을 다음 입력으로 사용
            dec_input = tf.expand_dims(targ[:, t], 1)
    
    # 5단계: 평균 loss 계산
    batch_loss = loss / int(targ.shape[1])
    
    # 6단계: 그래디언트 계산
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    
    # 7단계: 가중치 업데이트
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss

# --- 5.4 학습 실행 ---
EPOCHS = 100  # 에폭 수 (작은 데이터셋이므로 많이 학습)

print(" 학습 시작...\n")

for epoch in range(EPOCHS):
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    # 배치별 학습
    for (batch, (inp, targ)) in enumerate(dataset):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
    
    # 10 에폭마다 출력
    if (epoch + 1) % 10 == 0:
        avg_loss = total_loss / len(korean_tensor) * BATCH_SIZE
        print(f'Epoch {epoch+1:3d} | Loss: {avg_loss:.4f}')
        
        # 체크포인트 저장
        checkpoint.save(file_prefix=checkpoint_prefix)

print("\n 학습 완료!")

 학습 시작...

Epoch  10 | Loss: 2.0082
Epoch  20 | Loss: 1.9315
Epoch  30 | Loss: 1.7716
Epoch  40 | Loss: 1.6090
Epoch  50 | Loss: 1.3166
Epoch  60 | Loss: 1.0440
Epoch  70 | Loss: 0.8881
Epoch  80 | Loss: 0.7242
Epoch  90 | Loss: 0.6065
Epoch 100 | Loss: 0.4917

 학습 완료!


In [17]:
def evaluate(sentence):
    """
	한국어 문장을 영어로 번역
    
    Args:
        sentence: 입력 문장 (한국어)
    
    Returns:
        result: 번역된 문장 (영어)
        attention_weights: Attention 가중치 (시각화용)
    
    Greedy Decoding:
        매 스텝마다 가장 높은 확률의 단어 선택
        빠르지만 최적해 보장 안 됨
        
    Beam Search (고급):
        여러 후보를 동시에 탐색
        더 좋은 번역 가능하지만 느림
    """
    
    # 1단계: 문장 전처리
    sentence = preprocess_sentence(sentence, is_korean=True)
    
    # 2단계: 정수 시퀀스로 변환
    inputs = [korean_tokenizer.word_index.get(word, korean_tokenizer.word_index['<unk>'])
              for word in sentence.split()]
    
    # 3단계: 패딩 추가
    inputs = tf.keras.preprocessing.sequence.pad_sequences(
        [inputs],
        maxlen=max_korean_len,
        padding='post'
    )
    inputs = tf.convert_to_tensor(inputs)  # [1, seq_len]
    
    result = ''
    attention_plot = np.zeros((max_english_len, max_korean_len))
    
    # 4단계: Encoder 실행
    hidden = [tf.zeros((1, units)), tf.zeros((1, units))]
    enc_out, enc_h, enc_c = encoder(inputs, hidden)
    
    # 5단계: Decoder 초기화
    dec_hidden = enc_h
    dec_input = tf.expand_dims([english_tokenizer.word_index['<start>']], 0)
    
    # 6단계: 디코딩 루프
    for t in range(max_english_len):
        # 예측 수행
        predictions, dec_hidden, attention_weights = decoder(
            dec_input,
            dec_hidden,
            enc_out
        )
        
        # Attention 가중치 저장 (시각화용)
        attention_weights = tf.reshape(attention_weights, (-1,))
        attention_plot[t] = attention_weights.numpy()
        
        # 가장 높은 확률의 단어 선택 (Greedy)
        predicted_id = tf.argmax(predictions[0]).numpy()
        
        # 정수 → 단어 변환
        predicted_word = english_tokenizer.index_word.get(predicted_id, '<unk>')
        
        # <end> 토큰이면 중단
        if predicted_word == '<end>':
            break
        
        # 결과에 추가
        result += predicted_word + ' '
        
        # 예측 단어를 다음 입력으로 사용 (Teacher Forcing 없음)
        dec_input = tf.expand_dims([predicted_id], 0)
    
    return result.strip(), attention_plot

In [18]:
# 번역
# 추론할때 이전 예측을 다음 입력으로 사용
def evaluate(sentence):
    """
	한국어 문장을 영어로 번역
    
    Args:
        sentence: 입력 문장 (한국어)
    
    Returns:
        result: 번역된 문장 (영어)
        attention_weights: Attention 가중치 (시각화용)
    
    Greedy Decoding:
        매 스텝마다 가장 높은 확률의 단어 선택
        빠르지만 최적해 보장 안 됨
        
    Beam Search (고급):
        여러 후보를 동시에 탐색
        더 좋은 번역 가능하지만 느림
    """
    
    # 1단계: 문장 전처리
    sentence = preprocess_sentence(sentence, is_korean=True)
    
    # 2단계: 정수 시퀀스로 변환
    inputs = [korean_tokenizer.word_index.get(word, korean_tokenizer.word_index['<UNK>'])
              for word in sentence.split()]
    
    # 3단계: 패딩 추가
    inputs = tf.keras.preprocessing.sequence.pad_sequences(
        [inputs],
        maxlen=max_korean_len,
        padding='post'
    )
    inputs = tf.convert_to_tensor(inputs)  # [1, seq_len]
    
    result = ''
    attention_plot = np.zeros((max_english_len, max_korean_len))
    
    # 4단계: Encoder 실행
    hidden = [tf.zeros((1, units)), tf.zeros((1, units))]
    enc_out, enc_h, enc_c = encoder(inputs, hidden)
    
    # 5단계: Decoder 초기화
    dec_hidden = enc_h
    dec_input = tf.expand_dims([english_tokenizer.word_index['<start>']], 0)
    
    # 6단계: 디코딩 루프
    for t in range(max_english_len):
        # 예측 수행
        predictions, dec_hidden, attention_weights = decoder(
            dec_input,
            dec_hidden,
            enc_out
        )
        
        # Attention 가중치 저장 (시각화용)
        attention_weights = tf.reshape(attention_weights, (-1,))
        attention_plot[t] = attention_weights.numpy()
        
        # 가장 높은 확률의 단어 선택 (Greedy)
        predicted_id = tf.argmax(predictions[0]).numpy()
        
        # 정수 → 단어 변환
        predicted_word = english_tokenizer.index_word.get(predicted_id, '<unk>')
        
        # <end> 토큰이면 중단
        if predicted_word == '<end>':
            break
        
        # 결과에 추가
        result += predicted_word + ' '
        
        # 예측 단어를 다음 입력으로 사용 (Teacher Forcing 없음)
        dec_input = tf.expand_dims([predicted_id], 0)
    
    return result.strip(), attention_plot

In [19]:

korean_sentences[1]

'오늘 날씨가 좋아요'

In [20]:
english_tokenizer.word_index

{'<UNK>': 1,
 '<start>': 2,
 '<end>': 3,
 'is': 4,
 'the': 5,
 'i': 6,
 'a': 7,
 'this': 8,
 'hello': 9,
 'weather': 10,
 'nice': 11,
 'today': 12,
 'am': 13,
 'student': 14,
 'an': 15,
 'apple': 16,
 'cat': 17,
 'sleeping': 18,
 'will': 19,
 'it': 20,
 'rain': 21,
 'tomorrow': 22,
 'like': 23,
 'coffee': 24,
 'he': 25,
 'doctor': 26,
 'book': 27,
 'interesting': 28,
 'we': 29,
 'are': 30,
 'friends': 31}

In [21]:
# 번역 테스트 
for kor in korean_sentences:
    print(f'한국어 : {kor:20s} 번역 : {evaluate(kor)[0]}')

한국어 : 안녕하세요                번역 : this is an apple
한국어 : 오늘 날씨가 좋아요           번역 : the weather is nice today
한국어 : 저는 학생입니다             번역 : i am a student
한국어 : 이것은 사과입니다            번역 : this is an apple
한국어 : 고양이가 자고 있어요          번역 : the cat is sleeping
한국어 : 내일 비가 올까요            번역 : will it rain tomorrow
한국어 : 저는 커피를 좋아해요          번역 : i like coffee
한국어 : 그는 의사입니다             번역 : this is an apple
한국어 : 이 책은 재미있어요           번역 : this book is interesting
한국어 : 우리는 친구예요             번역 : i am a student


아, 지금 번역 결과를 보면 몇 가지 패턴이 보여요:

* **정확한 문장은 번역이 잘 됨**:
  `"오늘 날씨가 좋아요" → "the weather is nice today"`
  `"저는 학생입니다" → "i am a student"`

* **엉뚱하게 나오는 경우**:
  `"안녕하세요" → "this is an apple"`
  `"그는 의사입니다" → "this is an apple"`
  `"우리는 친구예요" → "i am a student"`

이런 현상이 나타나는 이유는 보통 다음과 관련 있습니다:

1. **데이터 부족 / 제한된 학습 데이터**

   * 학습 데이터에 `"안녕하세요"`나 `"그는 의사입니다"` 같은 문장이 없거나, 아주 적게 등장하면 모델이 유사한 패턴을 근거로 엉뚱하게 번역합니다.
   * 작은 데이터셋으로 학습했을 때, 모델은 **빈도 높은 문장 패턴**을 우선으로 예측합니다.
     예: `"this is an apple"`, `"i am a student"` 같은 반복되는 문장이 많으면 새로운 입력에 그걸 그대로 적용.

2. **모델 용량 부족 / 과적합**

   * LSTM 기반 작은 시퀀스-투-시퀀스 모델은 문장 길이와 복잡한 의미를 일반화하는 능력이 제한됩니다.
   * 특히 새로운 문장에서는 기존에 학습한 패턴만 따라가는 경향이 생깁니다.

3. **Teacher Forcing / 학습 방식 문제**

   * 학습 시 디코더가 항상 정답 다음 단어를 입력으로 받도록 했으면 모델이 새로운 문장을 생성하는 능력이 떨어집니다.
   * 추론 시 Greedy Decoding으로 매번 가장 높은 확률 단어를 선택하면, 학습 패턴에 편향된 출력이 나옵니다.

4. **단어 단위 토크나이저 한계**

   * 단어 단위 토크나이저로 학습하면 **희귀 단어**나 변형된 단어에 대한 일반화가 힘듭니다.
   * 예: `"커피를"` 같은 단어가 학습 데이터에 적으면 `"coffee"`로 올바르게 번역되지 않을 수 있습니다.

---

정리하면, 지금 번역이 엉뚱한 이유는:

* 학습 데이터가 적고 패턴이 제한적임
* 모델 용량/일반화 능력 부족
* 단어 단위 토크나이저의 희귀 단어 처리 문제
* Teacher Forcing / Greedy Decoding으로 인한 편향

원하면 제가 **이 모델로도 번역 품질을 어느 정도 개선할 수 있는 방법**을 단계별로 정리해줄 수 있습니다.

좋아요! 지금 LSTM 기반 시퀀스-투-시퀀스 모델에서 번역 품질을 어느 정도 개선할 수 있는 방법을 단계별로 정리해드릴게요.

---

## 1️⃣ 데이터 측면 개선

1. **데이터 양 늘리기**

   * 현재 예시를 보면 데이터가 적어서 패턴 편향이 심합니다.
   * 최소 수천 ~ 수만 문장 이상이 필요합니다.
   * 문장 다양성을 확보: 짧은 문장, 긴 문장, 다양한 주제 포함.

2. **Start/End 토큰 처리 확인**

   * 이미 `<start>`와 `<end>`를 분리했지만, 학습 시 모든 문장에 적용되어야 합니다.
   * 학습 데이터에서 `<start>`와 `<end>`를 누락하면 디코더가 시작/종료를 잘못 예측할 수 있음.

3. **정규화 및 토큰화**

   * 한국어와 영어 모두 일관된 토큰화 필요
   * 가능한 경우 한국어 형태소 분석 후 토큰화(Mecab, Kkma 등)

---

## 2️⃣ 모델 구조 개선

1. **모델 용량 늘리기**

   * LSTM units 수 증가: 예) 128 → 256 이상
   * 레이어 추가: 인코더/디코더 LSTM을 2층으로 쌓으면 복잡한 문장 처리 가능

2. **양방향 인코더 사용**

   * Bi-LSTM으로 문장 의미를 앞/뒤에서 모두 인코딩

3. **Attention 적용**

   * 이미 Attention이 있지만, 제대로 학습이 되는지 확인 필요
   * Attention 가중치 시각화로 모델이 어느 단어에 집중하는지 확인

---

## 3️⃣ 학습 방식 개선

1. **Teacher Forcing 비율 조절**

   * 너무 많이 사용하면 새로운 시퀀스 생성 시 일반화가 떨어집니다.
   * 일부는 이전 예측을 디코더 입력으로 사용하도록 섞어서 학습

2. **학습 Epoch와 Optimizer**

   * 충분한 epoch 학습 필요 (데이터 적으면 과적합 주의)
   * Learning rate 조절, Adam 사용

3. **Loss와 Metric 확인**

   * SparseCategoricalCrossentropy 사용
   * BLEU score나 정확도 모니터링

---

## 4️⃣ 추론 단계 개선

1. **Greedy Decoding → Beam Search**

   * Greedy는 항상 최대 확률 단어 선택 → 반복 패턴에 편향
   * Beam Search: 상위 N개의 후보 문장을 유지 → 더 자연스러운 번역 가능
   * Beam Width 3~5 정도 추천

2. **출력 후 후처리**

   * `<end>` 이후 단어 제거
   * 불필요한 반복 단어 제거

---

## 5️⃣ 토크나이저 개선 (Optional)

1. **단어 단위 → 서브워드(Byte Pair Encoding, SentencePiece)**

   * 희귀 단어 처리 가능
   * 형태소 변화에도 대응 가능
   * 데이터가 작아도 일반화가 잘 됨

---

💡 **정리**:

* 데이터 부족 → 데이터 늘리기 & 다양화
* 모델 부족 → LSTM 용량/층수 증가 + Bi-LSTM + Attention
* 학습 편향 → Teacher Forcing 조절 + 충분한 epoch
* 추론 편향 → Beam Search 사용
* 희귀 단어 → 서브워드 토크나이저

---


Q :
어느정도 개선할수있는 방법이라고 했는데 그럼 완전개선을 할려면어떻게 해야되?

좋아요, “어느 정도 개선”과 달리 **완전하게 번역 품질을 높이려면** 접근 방식 자체를 바꿔야 합니다. 현재 LSTM 기반 시퀀스-투-시퀀스 모델은 구조적 한계가 있어서 데이터가 많아도 한계가 있습니다. 완전 개선을 위해서는 다음 단계가 필요합니다.

---

## 1️⃣ 모델 아키텍처 변경 (핵심)

1. **Transformer 계열 모델 사용**

   * LSTM → **Transformer** (Attention 기반)
   * 장점:

     * 문장 길이 상관없이 병렬 처리 가능
     * Self-Attention으로 문맥 이해 뛰어남
     * 장문의 문장도 자연스럽게 번역
   * 대표 모델:

     * **Vanilla Transformer**: TensorFlow/Keras 구현 가능
     * **T5 / mT5 / BERT2BERT / MarianMT / mBART**: 이미 학습된 모델 활용 가능

2. **사전 학습된 모델 활용 (Pretrained NMT)**

   * 직접 학습하는 것보다, 대규모 데이터로 학습된 **Multilingual Translation 모델**을 활용
   * 예시: Hugging Face Transformers

     * `Helsinki-NLP/opus-mt-ko-en`
     * `mBART50` 등
   * 장점:

     * 소규모 데이터라도 바로 사용 가능
     * 문장 길이, 희귀 단어, 다양한 문체 대응

---

## 2️⃣ 데이터 측면

* **대규모 평행 코퍼스** 확보 필요

  * 한국어-영어 문장 쌍 수십만~수백만 문장
* **데이터 다양화**

  * 뉴스, 대화체, 문학, 기술문서 등
* **데이터 전처리**

  * 서브워드(BPE, SentencePiece) 토크나이저
  * 특수문자/숫자/약어 처리
* **증강(Augmentation)**

  * 역번역(back-translation)으로 데이터 확장 가능

---

## 3️⃣ 학습 방식 개선

* **Pretraining + Fine-tuning**

  * 이미 학습된 번역 모델을 가져와서, 내가 가진 데이터로 fine-tuning
* **Beam Search + Length Penalty**

  * 추론 시 Greedy 대신 Beam Search
  * 문장 길이 페널티 적용 → 더 자연스러운 문장
* **Ensemble**

  * 여러 모델의 예측을 결합하면 번역 품질 상승

---

## 4️⃣ 평가와 모니터링

* BLEU, ROUGE, METEOR 등 여러 지표 사용
* 단순 정확도보다 **문맥 자연스러움** 평가 필요
* 사람 평가(Human Evaluation)도 중요

---

### ✅ 결론

* LSTM 기반 시퀀스-투-시퀀스 모델로는 한계 존재
* 완전 개선을 위해서는 **Transformer + Pretrained NMT 모델 사용**이 현실적이고 효과적
* 학습 데이터, 토크나이저, 추론 방식까지 모두 바꿔야 고품질 번역 가능

---

원하면 제가 지금 **LSTM 모델 대신 Hugging Face의 사전 학습 모델**로 바로 한국어 → 영어 번역 테스트할 수 있는 코드를 만들어서 보여줄 수도 있어요.
그거 만들어드릴까요?
